In [162]:
import math
import itertools
import pandas

In [149]:
enum = { "nada": 0, "básico": 1, "médio": 2, "avançado": 3 }
inverse_enum = { 0: "nada", 1: "básico", 2: "médio", 3: "avançado" }
r = itertools.combinations_with_replacement([
    enum["nada"],
    enum["básico"],
    enum["médio"],
    enum["avançado"]
], 3)
r = list(r)

In [153]:
def format_result(i, sim):
    return {
        "Excel": inverse_enum[i[0]],
        "PPT": inverse_enum[i[1]],
        "Word": inverse_enum[i[2]],
        "Similaridade": sim
    }

## Sem match | Pesos iguais

$$S_{i} = \dfrac{\sum_{k = 0}^N \frac{r_{ik}}{|L| - 1}C^{r_{ik} - (|L| - 1)}}                                  {\sum_{k = 0}^N
			C^{r_{ik} - (|L| - 1)}}$$

$$ 0 < C < \frac{|L| - 1}{|L| - 2} $$

In [156]:
def no_match_equal_weights(i, C = [1.4, 1.4, 1.4]):
    numerator = 0
    denominator = 0
    
    for _i,  _c in list(zip(i, C)):
        denominator += _c**(_i-3)
        numerator += (_i/3)*_c**(_i-3)        
        
    return format_result(i, numerator/denominator)

## Sem match | Pesos diferentes

$$S_{ij} = \dfrac{\sum_{k = 0}^N \frac{r_{ik}}{|L| -1 } \cdot (1 + \frac{r_{jk}}{|L| -1 }) \cdot C^{r_{ik} - ({|L| -1 })}}                                  {\sum_{k = 0}^N (1 + \frac{r_{jk}}{|L| -1 })  \cdot  C^{r_{ik} - ({|L| -1 })}}$$

$$ 0 < C < \frac{|L| - 1}{|L| - 2} $$

In [155]:
def no_match_diff_weights(i, j = [3, 3, 3], C = [1.4, 1.4, 1.4]):
    numerator = 0
    denominator = 0
    
    for _i, _j, _c in list(zip(i, j, C)):
        denominator += ((_j/3)+1)*_c**(_i-3)
        numerator += ((_i/3)*((_j/3)+1))*_c**(_i-3)        
        
    return format_result(i, numerator/denominator)

## Com match | Pesos diferentes

$$S_{ij} = \dfrac{\sum_{k = 0}^N \frac{r_{ik} - r_{jk}}{|L| -1 } \cdot (1 + \frac{r_{jk}}{|L| -1 }) \cdot C^{r_{ik} - ({|L| -1 })}}                                  {\sum_{k = 0}^N (1 + \frac{r_{jk}}{|L| -1 })  \cdot  C^{r_{ik} - ({|L| -1 })}}$$

$$ 0 < C < \frac{|L| - 1}{|L| - 2} $$

$$ C = \begin{cases} C, & \mbox{if } r_{ik} \leq r_{jk} \\ \frac{1}{C}, & \mbox{if } r_{ik} > r_{jk} \end{cases} $$

In [157]:
def with_match_diff_weights(i, j = [3, 3, 3], C = [1.4, 1.4, 1.4]):
    numerator = 0
    denominator = 0
    
    for _i, _j, _c in list(zip(i, j, C)):
        if _i > _j:
            _c = 1/_c
        denominator +=  ((_j/3)+1)*_c**(_i-3)
        numerator += ((_i - _j)/3)*((_j/3)+1)*_c**(_i-3)

    return format_result(i, numerator/denominator)

# Generalizações


### Sem match | Pesos iguais

1. Para qualquer valor de $ i_k $, $i_k + 1$ sempre terá um valor de similaridade maior que $i_k$;
2. Dado um valor de $i_k$ menor que $|L|- 1$, $C + 1$ sempre resultará em um valor de similaridade maior que $C$;
3. Independente do valor de $C$, se $i$ for igual à $|L| - 1$, todos os valores de similaridade serão iguais;
4. $S_i \in	[0, 1]$

### Sem match | Pesos diferentes

1. Para um mesmo valor de $ i_k $ entre objetos, um objeto com $ j_k + 1 $ resultará em um valor de similaridade menor que $j_k$. *Motivo: ao aumentar o peso de $j_k$, conhecimentos menores são mais "penalizados" e portanto resultam em similaridades menores*;
2. Para um mesmo valor de $j_k$ entre objetos, um objeto com $i_k + 1$ resultará em um valor de similaridade maior do que um objeto com $j_k$;
3. Para um objeto com $i_k$, $j_k$ e $C$ e outro com $i_k$, $j_k$ e $C + 1$, o primeiro terá um valor de similaridade menor que o segundo;
4. Dados objetos com os mesmos valores de $i_k$, para um objeto com todos $j_k$ iguais, um objeto com todos $j_k + 1$ resultará em um valor de similaridade igual ao objeto anterior;
5. Para um objeto com todos $i_k$ iguais, o valor de similaridade será o mesmo independente do $j_k$ e $C$;
6. $S_{ij} \in [0, 1]$

### Com match | Pesos diferentes

*Herda as generalizações 1, 2 e 3 de Sem match | Pesos diferentes*

1. Dado que $i_k$ de um objeto é igual à $j_k$, o valor de similaridade será $0$
2. Dado que $i_k$ de um objeto é menor que $j_k$, o valor de similaridade será no intervalo $]-1, 0[$
3. Dado que $i_k$ de um objeto é maior que $j_k$, o valor de similaridade será no intervalo $]0, 1[$
4. $S_{ij} \in	 [-1,1]$, porém ocupará um subintervalo de módulo igual à $1$, por exemplo, de $-0.55$ a $0.45$ e $-0.9$ a $0.1$

------------------------

### Pontos importantes

- Por que em Com match | Pesos diferentes a constante $C$ é invertido quando $i_k$ é maior que $j_k$?

Por causa da possibilidade de um valor negativo. Antes de chegar na inversão, a seguinte tabela definia o comportamento do mérito e o valor de `C`.

Supondo que $i_k = 1$

- | Match negativo | Match positivo | $$ \Rightarrow $$ | Match negativo | Match positivo 
--- | --- | --- | --- | --- | --- 
C grande | $$ -(5^{-2}) = -0.04 $$ | $$ 5^{-2} = 0.04 $$ | $$ \Rightarrow $$ | bastante mérito | pouco mérito
C pequeno | $$ -(0.7^{-2}) = -2.04 $$ | $$ 0.7^{-2} = 2.04 $$ | $$ \Rightarrow $$ | pouco mérito | bastante mérito


Se o match é negativo e o valor de $C$ for alto, então o mérito é alto também, porém isso não ocorre quando o match é positivo e sim o oposto, por isso o valor de $C$ é invertido quando $i_k$ é maior que $j_k$. Dessa forma, o usuário não precisa inverter manualmente. Assim chega-se na seguinte tabela:

- | Match negativo | Match positivo | $$ \Rightarrow $$ | Match negativo | Match positivo 
--- | --- | --- | --- | --- | --- 
C grande | $$ -(5^{-2}) = -0.04 $$ | $$ \frac{1}{5}^{-2} = 25 $$ | $$ \Rightarrow $$ | bastante mérito | bastante mérito
C pequeno | $$ -(0.7^{-2}) = -2.04 $$ | $$ \frac{1}{0.7}^{-2} = 0.49 $$ | $$ \Rightarrow $$ | pouco mérito | pouco mérito


# Tests

#### Candidate (i)

Excel | PPT | Word
--- | --- | ---
0 | 2 | 3
3 | 2 | 0
0 | 3 | 3
1 | 2 | 2
0 | 0 | 0
1 | 1 | 1
2 | 2 | 2
3 | 3 | 3
0 | 2 | 2
3 | 2 | 2

#### Company (j)

Excel | PPT | Word
--- | --- | ---
0 | 2 | 3
3 | 2 | 0
0 | 3 | 3
1 | 2 | 2
0 | 0 | 0
1 | 1 | 1
2 | 2 | 2
3 | 3 | 3
0 | 2 | 2
3 | 2 | 2

In [170]:
i_s = [
    [0 , 2 , 3],
    [3 , 2 , 0],
    [0 , 3 , 3],
    [1 , 2 , 2],
    [0 , 0 , 0],
    [1 , 1 , 1],
    [2 , 2 , 2],
    [3 , 3 , 3],
    [0 , 2 , 2],
    [3 , 2 , 2]
]


j_s = []
for i in i_s:
    for j in i_s:
       j_s.append([i, j])

In [165]:
result = sorted(list(map(no_match_equal_weights, i_s)), key=lambda x: x["Similaridade"])
df = pandas.DataFrame(result)
df

,Excel,PPT,Word,Similaridade
0,nada,nada,nada,0.000000
1,básico,básico,básico,0.333333
2,nada,médio,médio,0.531165
3,básico,médio,médio,0.578947
4,médio,médio,médio,0.666667
5,nada,médio,avançado,0.710145
6,avançado,médio,nada,0.710145
7,avançado,médio,médio,0.803922
8,nada,avançado,avançado,0.845869
9,avançado,avançado,avançado,1.000000


In [215]:
with_match_diff_weights([1, 1, 1], [1, 1, 1], [1.4, 1.4, 1.4])

{'Excel': 'básico', 'PPT': 'básico', 'Word': 'básico', 'Similaridade': 0.0}

In [214]:
with_match_diff_weights([1, 1, 1], [2, 2, 2], [1.4, 1.4, 1.4])

{'Excel': 'básico',
 'PPT': 'básico',
 'Word': 'básico',
 'Similaridade': -0.33333333333333326}